In [2]:
import models
import numpy as np
import jax.numpy as jnp
import objax
import jax
import data
import util
from sklearn.metrics import confusion_matrix

## run test data and check logits for a sample

In [2]:
# best simo model

net = models.EnsembleNet(num_models=4,
                         num_classes=10,
                         dense_kernel_size=51,
                         seed=0)
objax.io.load_var_collection('saved_models/simo_12/20200909_031227/final.npz', net.vars())

In [3]:
# best mimo model

net = models.EnsembleNet(num_models=2,
                         num_classes=10,
                         dense_kernel_size=57,
                         seed=0)
objax.io.load_var_collection('saved_models/mimo_01/20200909_160639/final.npz', net.vars())

In [4]:
imgs, labels = data.test_dataset()
imgs.shape, labels.shape

((2700, 64, 64, 3), (2700,))

In [5]:
S = 500

sample_imgs = imgs #[:S]
sample_labels = labels #[:S]

In [6]:
logits = net.logits(sample_imgs, single_result=False)
logits.shape

(2, 2700, 10)

overall ensemble predictions and accuracy

In [7]:
ensemble_predictions = jnp.argmax(logits.sum(axis=0), axis=-1)
#ensemble_predictions

In [8]:
#sample_labels

In [9]:
util.accuracy(ensemble_predictions, sample_labels)

0.9081481481481481

In [10]:
confusion_matrix(sample_labels, ensemble_predictions)

array([[289,   2,   1,   2,   0,   2,   7,   0,   4,   0],
       [  1, 290,   3,   0,   0,   4,   0,   0,   1,   1],
       [  4,   6, 232,   4,   2,   6,  39,   1,   2,   0],
       [  3,   0,   2, 193,  11,   0,   7,   4,   1,   0],
       [  0,   0,   2,   2, 250,   0,   2,   6,   0,   0],
       [  5,   9,   8,   0,   0, 191,   2,   0,   0,   1],
       [  4,   0,  15,   1,   1,   5, 222,   2,   1,   0],
       [  0,   0,   6,   3,   7,   0,   1, 279,   0,   0],
       [  8,   0,   1,  18,   1,   3,   3,   0, 216,   0],
       [  1,   4,   0,   1,   0,   2,   0,   0,   3, 290]])

per model predictions and accuracy

In [13]:
for m in range(2):
    print("model", m)
    single_model_predictions = jnp.argmax(logits[m], axis=-1)
    single_model_accuracy = util.accuracy(single_model_predictions, sample_labels)
    print("single_model_accuracy %0.3f" % single_model_accuracy)
    print(confusion_matrix(sample_labels, single_model_predictions))

model 0
single_model_accuracy 0.880
[[271   2   5   4   0   6  14   0   5   0]
 [  0 285   4   0   0  10   0   0   1   0]
 [  1   5 212   5   3   5  59   2   3   1]
 [  4   0   3 179  17   1  11   6   0   0]
 [  0   0   0   4 247   0   1  10   0   0]
 [  2   6   9   0   0 195   4   0   0   0]
 [  3   0  15   4   1   6 217   2   3   0]
 [  0   0   6   1   3   0   1 285   0   0]
 [ 11   0   2  28   2   5   4   0 198   0]
 [  2   7   0   1   0   3   0   0   2 286]]
model 1
single_model_accuracy 0.894
[[293   2   1   1   0   0   7   0   2   1]
 [  0 291   3   0   0   1   0   0   4   1]
 [ 10   6 228   6   2   7  27   3   7   0]
 [  5   0   0 194  14   0   2   3   3   0]
 [  0   0   0   3 252   0   3   4   0   0]
 [  7  11  10   1   0 183   0   0   3   1]
 [  8   1  22   6   3   6 202   2   1   0]
 [  0   0   4   5  15   0   1 271   0   0]
 [ 12   0   2  15   1   1   3   0 216   0]
 [  5   4   3   0   0   2   0   0   4 283]]
